# InvokeAI GUI on Google Colab (GPU)

This notebook installs InvokeAI from this repository and launches the web UI behind a Cloudflare tunnel so you can open it from your browser.

## 1) Runtime setup
In Colab, set **Runtime -> Change runtime type -> T4/A100 GPU** before running.

In [ ]:
!nvidia-smi

In [ ]:
import os

REPO_URL = "https://github.com/invoke-ai/InvokeAI.git"
BRANCH = "main"  # change this if you want a different branch/tag
WORKDIR = "/content/InvokeAI"
INVOKE_ROOT = "/content/invokeai"
PORT = 9090

if not os.path.exists(WORKDIR):
    !git clone --depth 1 --branch {BRANCH} {REPO_URL} {WORKDIR}
else:
    print(f"Using existing repo at {WORKDIR}")


## 2) Install dependencies and InvokeAI

In [ ]:
%cd /content/InvokeAI

# Install uv and resolve its binary path robustly in Colab
!curl -LsSf https://astral.sh/uv/install.sh | sh
!UV_BIN=$(command -v uv || true); \
if [ -z "$UV_BIN" ] && [ -x /usr/local/bin/uv ]; then UV_BIN=/usr/local/bin/uv; fi; \
if [ -z "$UV_BIN" ] && [ -x /root/.local/bin/uv ]; then UV_BIN=/root/.local/bin/uv; fi; \
if [ -z "$UV_BIN" ]; then echo 'uv not found after install' && exit 1; fi; \
echo "Using uv at: $UV_BIN"; \
$UV_BIN venv .venv --python 3.11

# Install InvokeAI into the venv (CUDA backend for Colab GPU)
!UV_BIN=$(command -v uv || true); \
if [ -z "$UV_BIN" ] && [ -x /usr/local/bin/uv ]; then UV_BIN=/usr/local/bin/uv; fi; \
if [ -z "$UV_BIN" ] && [ -x /root/.local/bin/uv ]; then UV_BIN=/root/.local/bin/uv; fi; \
$UV_BIN pip install --python .venv/bin/python -e . --torch-backend=cu128

# Verify CLI is available
!.venv/bin/python -m invokeai --version || true
!ls -l .venv/bin/invokeai-web


## 3) Start InvokeAI server

In [ ]:
import os

os.makedirs(INVOKE_ROOT, exist_ok=True)
os.environ["INVOKEAI_ROOT"] = INVOKE_ROOT
os.environ["INVOKEAI_HOST"] = "0.0.0.0"
os.environ["INVOKEAI_PORT"] = str(PORT)

%cd /content/InvokeAI
# Start via python -m invokeai.app.run_app to avoid PATH/script issues
!nohup .venv/bin/python -m invokeai.app.run_app --root {INVOKE_ROOT} > /content/invokeai.log 2>&1 &
!sleep 12
!tail -n 40 /content/invokeai.log


## 4) Create a public URL (Cloudflare Tunnel)

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O /tmp/cloudflared.deb
!dpkg -i /tmp/cloudflared.deb
!nohup cloudflared tunnel --url http://127.0.0.1:{PORT} --no-autoupdate > /content/cloudflared.log 2>&1 &
!sleep 8
!grep -o 'https://[-a-zA-Z0-9]*\.trycloudflare.com' /content/cloudflared.log | head -n 1


## 5) Keep-alive / Logs
If the UI does not load, re-run log cells and verify the URL is printed.

In [ ]:
!tail -n 80 /content/invokeai.log
!tail -n 40 /content/cloudflared.log